# Лекция №8: Практика ООП: Проектирование и реализация классов

### Цели лекции:
1.  **Углубить понимание инкапсуляции:** Научиться управлять доступом к атрибутам с помощью геттеров, сеттеров и свойств (properties).
2.  **Изучить основные "магические" методы:** Понять назначение `__init__`, `__str__` и `__del__`.
3.  **Научиться проектировать классы:** Рассмотреть полный цикл создания класса от идеи до демонстрационной программы.
4.  **Освоить работу с коллекциями объектов:** Понять, как хранить объекты в списках и взаимодействовать с ними.

## Часть 1. Краткое повторение

На прошлой лекции мы узнали, что:
- **Класс** — это чертеж.
- **Объект** — это экземпляр, созданный по чертежу.
- `__init__(self, ...)` — это **конструктор**, который инициализирует атрибуты объекта при его создании.
- `self` — это ссылка на сам объект внутри класса.
- **Атрибуты** — это данные объекта (переменные).
- **Методы** — это поведение объекта (функции).

Сегодня мы углубимся в практические аспекты создания классов.

## Часть 2. Инкапсуляция на практике: Геттеры и Сеттеры

Инкапсуляция — это не только объединение данных и методов, но и защита этих данных от некорректного изменения. Рассмотрим класс для товара в интернет-магазине.

In [ ]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

book = Product("Война и мир", 5000)
print(f"{book.name} стоит {book.price} тг.")

# Проблема: любой может присвоить некорректное значение
book.price = -1000
print(f"Новая цена: {book.price} тг. -> Это бессмысленно!")

Чтобы решить эту проблему, мы должны скрыть атрибут и предоставить специальные методы для работы с ним.

- **Соглашение об именовании:** Атрибут, который не предназначен для прямого доступа извне, именуется с одним подчеркиванием в начале (например, `_price`). Это "защищенный" атрибут.
- **Сеттер (setter):** Метод для **установки** значения атрибута. В нем мы можем добавить логику проверки (валидации).
- **Геттер (getter):** Метод для **получения** значения атрибута.

In [ ]:
class ProductWithValidation:
    def __init__(self, name, price):
        self.name = name
        self._price = price # Используем защищенный атрибут

    # Геттер для цены
    def get_price(self):
        return self._price

    # Сеттер для цены
    def set_price(self, new_price):
        if new_price > 0:
            self._price = new_price
            print("Цена успешно обновлена.")
        else:
            print("Ошибка: цена должна быть положительной!")

phone = ProductWithValidation("Смартфон X", 150000)

# Получаем цену через геттер
print(f"Текущая цена: {phone.get_price()}")

# Пытаемся установить некорректную цену через сеттер
phone.set_price(-5000)

# Устанавливаем корректную цену
phone.set_price(145000)
print(f"Новая цена: {phone.get_price()}")

### Часть 2.1. Уровни доступа и декораторы @property

В Python нет строгих ключевых слов `private` или `protected`, как в Java или C++. Управление доступом основано на **соглашениях об именовании**.

1.  `self.price` — **Public (Публичный)**. Атрибут доступен для чтения и записи откуда угодно. Это стандартное поведение.
2.  `self._price` — **Protected (Защищенный)**. Одно подчеркивание в начале — это **сигнал** для других программистов: "Этот атрибут является внутренней частью класса. Вы можете его использовать, но лучше не меняйте его напрямую, если не уверены в своих действиях". Технически доступ к нему никак не ограничен.
3.  `self.__price` — **Private (Приватный)**. Два подчеркивания в начале включают механизм **искажения имен (name mangling)**. Python автоматически переименовывает этот атрибут в `_ИмяКласса__price`. Это делает случайный доступ к нему извне практически невозможным, обеспечивая более надежную инкапсуляцию.

In [ ]:
class TestAccess:
    def __init__(self):
        self.public = "Я публичный"
        self._protected = "Я защищенный"
        self.__private = "Я приватный"

t = TestAccess()
print(t.public)
print(t._protected)

try:
    print(t.__private) # Это вызовет ошибку!
except AttributeError as e:
    print(f"Ошибка: {e}")

# Но мы все еще можем получить доступ, если знаем искаженное имя
print(f"Доступ через искаженное имя: {t._TestAccess__private}")

### Pythonic-способ: Декораторы `@property` и `@setter`

Вызывать методы `get_price()` и `set_price()` функционально, но не очень удобно. Хотелось бы сохранить простоту `product.price`, но с валидацией. Для этого в Python существуют **декораторы**.

-   `@property` — превращает метод в **геттер**, который выглядит как обычный атрибут (вызывается без скобок).
-   `@<имя>.setter` — превращает метод в **сеттер**, который вызывается при попытке присвоить значение этому "атрибуту".

In [ ]:
class ProductPythonic:
    def __init__(self, name, price):
        self.name = name
        self.__price = price # Используем приватный атрибут для хранения

    @property
    def price(self):
        """Это геттер. Вызывается, когда мы читаем `obj.price`."""
        print("(Вызван геттер)")
        return self.__price

    @price.setter
    def price(self, new_price):
        """Это сеттер. Вызывается, когда мы пишем `obj.price = value`."""
        print("(Вызван сеттер)")
        if new_price > 0:
            self.__price = new_price
        else:
            print("Ошибка: цена должна быть положительной!")

# --- Демонстрация работы ---
tv = ProductPythonic("Телевизор 4K", 250000)

# Читаем значение. Вызывается метод, помеченный @property
current_price = tv.price 
print(f"Текущая цена: {current_price}")

# Пытаемся присвоить значение. Вызывается метод, помеченный @price.setter
print("\nПопытка установить некорректную цену...")
tv.price = -100

print("\nПопытка установить корректную цену...")
tv.price = 240000
print(f"Новая цена: {tv.price}")

## Часть 3. "Магические" методы (Dunder Methods)

Методы с двумя подчеркиваниями в начале и в конце (например, `__init__`) называются "магическими" или dunder-методами (от Double Underscore). Они выполняют специальные функции в жизненном цикле объекта.

### `__init__` — Конструктор
Мы уже с ним знакомы. Вызывается **в момент создания** объекта для его инициализации.

### `__str__` — Строковое представление объекта

Что произойдет, если мы попробуем напечатать наш объект `phone`?

In [ ]:
print(phone)

Мы видим неинформативное сообщение. Метод `__str__` позволяет определить, как объект будет представлен в виде строки, когда его пытаются напечатать (`print()`) или преобразовать в строку (`str()`).

In [ ]:
class ProductWithStr:
    def __init__(self, name, price):
        self.name = name
        self._price = price
    
    # ... (геттеры и сеттеры опущены для краткости) ...
    
    # Определяем строковое представление
    def __str__(self):
        # Этот метод ДОЛЖЕН возвращать строку
        return f"Товар: {self.name}, Цена: {self._price} тг."

laptop = ProductWithStr("Ноутбук Pro", 550000)
print(laptop) # Теперь вывод будет красивым и информативным

### `__del__` — Деструктор

Деструктор — это метод, который вызывается **непосредственно перед уничтожением объекта** сборщиком мусора Python. Его используют редко, в основном для освобождения внешних ресурсов (например, закрытия файла или сетевого соединения, открытого объектом).

> **Важно:** В Python вы не можете точно контролировать, когда именно будет вызван `__del__`. Это зависит от сборщика мусора. Поэтому на него не стоит полагаться для критически важной логики.

In [ ]:
class TemporaryObject:
    def __init__(self, name):
        self.name = name
        print(f"Объект '{self.name}' создан.")
    
    def __del__(self):
        print(f"Объект '{self.name}' уничтожается! Прощай, мир!")

def create_temp_object():
    print("--- Входим в функцию ---")
    temp = TemporaryObject("Временный")
    print("--- Выходим из функции ---")

create_temp_object()
# После выхода из функции, объект 'temp' становится недоступным,
# и сборщик мусора его уничтожает, вызывая __del__

## Часть 4. Работа с несколькими объектами

Сила ООП раскрывается при работе с множеством однотипных объектов. Их удобно хранить в списке или словаре.

**Задача:** Создать каталог товаров, найти самый дорогой товар и посчитать общую стоимость всех товаров на складе.

In [ ]:
# Используем наш класс ProductWithStr
catalog = [
    ProductWithStr("Ноутбук Pro", 550000),
    ProductWithStr("Смартфон X", 150000),
    ProductWithStr("Наушники Air", 85000)
]

print("--- Каталог товаров ---")
for product in catalog:
    print(product) # Используется __str__

# --- Анализ каталога ---
total_cost = 0
most_expensive_product = None

for product in catalog:
    total_cost += product._price # Здесь для простоты обращаемся напрямую
    
    if most_expensive_product is None or product._price > most_expensive_product._price:
        most_expensive_product = product

print(f"\nОбщая стоимость товаров: {total_cost} тг.")
print(f"Самый дорогой товар: {most_expensive_product}")

## Итог

На этой лекции мы перешли от теории к практике:
1.  Научились защищать данные с помощью **геттеров и сеттеров**, а также их элегантной альтернативы в виде `@property`.
2.  Разобрали **магические методы**, которые делают наши классы более удобными и предсказуемыми (`__str__`, `__del__`).
3.  Увидели, как можно эффективно работать со **списками объектов**, применяя уже знакомые нам циклы.

Теперь вы готовы к выполнению лабораторной работы, где вам предстоит самостоятельно спроектировать и реализовать классы по заданной спецификации.